**GluonCV Demo: Instance Segmentation using Webcam**

In [0]:
#@title
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  # display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

# 1. Install MXNet and GluonCV, load Python Packages
! pip install -q mxnet-mkl gluoncv --progress-bar off
from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt

# 2. Load Mask R-CNN Pre-trained Model
MaskRCNN = model_zoo.get_model('mask_rcnn_resnet50_v1b_coco', pretrained=True)

# 3. Transform the Image File to Suitable Format
new_data, new_image = data.transforms.presets.rcnn.load_test(filename)

# 4. Apply Mask R-CNN to do Instance Segmentation
box_ids, scores, bboxes, masks = MaskRCNN(new_data)

# 5. Display the Instance Segmentation Result
width, height = new_image.shape[1], new_image.shape[0]
new_masks, _ = utils.viz.expand_mask(masks[0], bboxes[0], (width, height), scores[0])
new_image = utils.viz.plot_mask(new_image, new_masks)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
# Threshold = 0.5 means we only display those bounding boxes with scores >= 0.5
ax = utils.viz.plot_bbox(new_image, bboxes[0], scores[0], box_ids[0], class_names=MaskRCNN.classes, thresh=0.5, ax=ax)
plt.show()

# 6. Print out the Numerical Result of Instance Segmentation
for i in range(len(box_ids[0])):
    if box_ids[0][i] != -1 and scores[0][i].asscalar() >= 0.5:
        print('Class =', MaskRCNN.classes[int(box_ids[0][i].asscalar())], ' Bounding Box =', bboxes[0][i].asnumpy(), ' Score =', scores[0][i].asscalar())